In [58]:
import sys
import os

from dotenv import load_dotenv

load_dotenv()

# Add the parent directory (project root) to sys.path
sys.path.append(os.path.abspath('..'))


In [2]:
from e2b_desktop import Sandbox, CommandExitException
E2B_API_KEY = os.environ.get("E2B_API_KEY")

In [3]:
# automatically load changes from the sandbox_steps module
%load_ext autoreload
%autoreload 2

In [4]:
from llm import BaseFunction, OpenAIModel, MemoryManager, GroqModel
import base64

In [5]:
class MoveFunction(BaseFunction):
    """
    Let's you move around in CS 1.6. 
    """
    function_schema = {
        "type": "function",
        "function": {
            "name": "move",
            "description": "Move in game in a specified direction.",
            "parameters": {
                "type": "object",
                "properties": {
                    "command": {
                        "type": "string",
                        "description": "Direction (forward, backward, left, right)."
                    },
                    "duration": {
                        "type": "number",
                        "description": "Duration in seconds for how long you should move."
                    }
                },
                "required": ["command", "duration"],
                "additionalProperties": False
            },
            "strict": True
        }
    }

    def __init__(self, desktop: Sandbox):
        self.desktop = desktop

    def execute(self, command: str, duration: float = None):
        print(f"Executing tool call: {command}")
        if command == "forward":
            self.desktop.write("wwwwwwwwwwwwww")
        elif command == "backward":
            self.desktop.write("ssssssssssssss")
        elif command == "left":
            self.desktop.write("aaaaaaaaaaaaaa")
        elif command == "right":
            self.desktop.write("dddddddddddddd")
        return f"Moved {command}"

In [92]:
# With custom configuration
desktop = Sandbox(
    display=":0",  # Custom display (defaults to :0)
    resolution=(1920, 1080),  # Custom resolution
    timeout = 3600) 

desktop.stream.start()

# Get stream URL
url = desktop.stream.get_url()
print(url)

https://6080-i4ec87re37jtoqjdsnbk7-59d5f765.e2b.app/vnc.html?autoconnect=true&resize=scale


In [8]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def get_screenshot(desktop: Sandbox):
    image_bytes = desktop.screenshot(format="bytes")

    # Save for debugging
    with open("debug_screenshot.jpg", "wb") as f:
        f.write(image_bytes)

    base64_image = base64.b64encode(image_bytes).decode("utf-8")

    return base64_image

def get_screenshot_message(desktop: Sandbox):
    base64_image = get_screenshot(desktop)
    return [{
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": f"The current situation on the screen",
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
                "detail": "low"
            },
        },
    ],
    }]

In [9]:
tools = {
        "move": MoveFunction(desktop)
        }

# llm = OpenAIModel(tools=tools,
#                   model="gpt-4.1-nano-2025-04-14")

llm = GroqModel(
                  model="meta-llama/llama-4-maverick-17b-128e-instruct")

memory = MemoryManager()
system_message = memory.get_memory_as_messages()

# user_message = [
#         {
#         "role": "user",
#         "content": [
#                 {
#                 "type": "text",
#                 "text": "You are playing cs go. Choose you next action using the function calling. Output only a function call, nothing else.",
#                 },
#                 ],
#         }
#              ]

user_message = [
        {
        "role": "user",
        "content": [
                {
                "type": "text",
                "text": "You are playing cs go. Choose your nexxt action.",
                },
                ],
        }
             ]




In [10]:
payload

NameError: name 'payload' is not defined

In [71]:
running_sandboxes = Sandbox.list()
running_sandboxes

[]

In [ ]:
# Sandbox.kill("ies3dctuy7o376e5yq5ex-16931f9b")

True

In [ ]:
Sandbox.left_click(100, 100) 
Sandbox.right_click(200, 200)

In [80]:
def install_csgo(desktop: Sandbox):
    desktop.open("https://www.cybersports.lt/setup/") # open the link in the default browser
    desktop.commands.run("sudo dpkg --add-architecture i386")
    desktop.commands.run("sudo apt update")
    desktop.wait(100)
    desktop.commands.run("sudo apt install -y wine32")
    desktop.wait(2000)
    try:
        desktop.commands.run("wine .") # this throws an error for fun
    except CommandExitException:
        pass
    desktop.open("https://drive.google.com/u/0/uc?id=1TIsvGACSrQOr1tgPaVpJebH375LjLIV6&export=download")
    desktop.wait(ms=4000)
    desktop.left_click(460, 305) #click the Download button
    desktop.wait(ms=7000) 
    desktop.commands.run("unzip Downloads/Windows7DefaultFonts.zip -d Downloads") # couldnt find
    desktop.commands.run("mv Downloads/Windows7DefaultFonts/* ~/.wine/drive_c/windows/Fonts/")
    desktop.left_click(880, 1040) # open the terminal (I know, clean)
    desktop.wait(100)
    desktop.left_click(1800, 800) # click on the terminal window
    desktop.write("wine Downloads/Counter-Strike-1.6-original.exe", chunk_size=50, delay_in_ms=25) 
    desktop.wait(2000)
    desktop.press("enter") # start the exe with wine
    desktop.wait(6000)
    desktop.press("enter") # start the installation
    desktop.wait(2000)
    desktop.press("enter") # one menu
    desktop.wait(2000)
    desktop.press("enter") # second menu
    desktop.wait(2000)
    desktop.press("enter") # final install button
    desktop.wait(30000) # installation process
    desktop.wait(30000) # installation process
    desktop.press("enter") # LAUNCH


In [93]:
install_csgo(desktop=desktop)

KeyboardInterrupt: 

In [48]:
desktop.open("https://www.cybersports.lt/setup/")

In [53]:
desktop.commands.run("wine .wine/drive_c/Games/'Counter-strike 1.6 Original'/hlds.exe")

KeyboardInterrupt: 

In [55]:
desktop.commands.run("sudo update-alternatives --set ip6tables /usr/sbin/ip6tables-legacy")

CommandResult(stderr='', stdout='update-alternatives: using /usr/sbin/ip6tables-legacy to provide /usr/sbin/ip6tables (ip6tables) in manual mode\n', exit_code=0, error='')

In [86]:
screenshot_message = get_screenshot_message(desktop)
payload = system_message + user_message + screenshot_message


In [97]:
response, messages = llm.complete(messages=payload)
print(response)

BadRequestError: Error code: 400 - {'error': {'message': "'messages.3' : for 'role:assistant' the following must be satisfied[('messages.3.tool_calls' : Value is not nullable)]", 'type': 'invalid_request_error'}}

In [76]:
desktop.kill()

In [57]:
host = desktop.get_host(3000) 
print(f'https://{host}')

https://3000-ieyqetuwsyrumezvppnwt-1f87b594.e2b.app


# CS server

In [63]:
desktop.commands.run("mkdir ~/SteamCMD")

CommandResult(stderr='', stdout='', exit_code=0, error='')

In [69]:
desktop.commands.run("touch ~/hlds/cstrike/banned.cfg")

CommandResult(stderr='', stdout='', exit_code=0, error='')

In [ ]:
desktop.commands.run("wget http://media.steampowered.com/client/steamcmd_linux.tar.gz")

CommandResult(stderr="--2025-05-05 22:36:58--  http://media.steampowered.com/client/steamcmd_linux.tar.gz\nResolving media.steampowered.com (media.steampowered.com)... 23.46.228.74, 23.46.228.75\nConnecting to media.steampowered.com (media.steampowered.com)|23.46.228.74|:80... connected.\nHTTP request sent, awaiting response... 200 OK\nLength: 2428561 (2.3M) [application/octet-stream]\nSaving to: 'steamcmd_linux.tar.gz'\n\n     0K .......... .......... .......... .......... ..........  2% 25.9M 0s\n    50K .......... .......... .......... .......... ..........  4% 6.66M 0s\n   100K .......... .......... .......... .......... ..........  6% 7.65M 0s\n   150K .......... .......... .......... .......... ..........  8% 29.2M 0s\n   200K .......... .......... .......... .......... .......... 10% 21.9M 0s\n   250K .......... .......... .......... .......... .......... 12% 20.1M 0s\n   300K .......... .......... .......... .......... .......... 14% 24.0M 0s\n   350K .......... .......... ....

In [75]:
# run the server: ./hlds_run -console -game cstrike +port 27015 +map de_dust2 +maxplayers 16 -pingboost 1